# This is how to create service tasks in Amazon ECS

In [2]:
import boto3

In [3]:
client = boto3.client('ecs')

# we created a cluster called cloudbook in the ecs portal
if we didn't, then go to the portal and do so.  instructions are in the book.

let's see if we can find it

In [4]:
client.list_clusters()['clusterArns']

[u'arn:aws:ecs:us-west-2:066301190734:cluster/default',
 u'arn:aws:ecs:us-west-2:066301190734:cluster/cloudbook']

Next let's see how many VMs (called containers in ecs) we have.  there shoud be two.

In [5]:
instance_list = client.list_container_instances(cluster='cloudbook')['containerInstanceArns']

In [6]:
instance_list

[u'arn:aws:ecs:us-west-2:066301190734:container-instance/1976abce-a701-48d4-8d47-36203ec37030',
 u'arn:aws:ecs:us-west-2:066301190734:container-instance/6db24ef9-cd3d-484c-ae8d-68222ccdf553']

we can even get their IP addresses

In [25]:
ec2instances = [ client.describe_container_instances(
        cluster='cloudbook',
        containerInstances=[instance]
    )[u'containerInstances'][0][u'ec2InstanceId'] for instance in instance_list]


In [28]:
ec2 = boto3.resource('ec2')
instances = ec2.instances.filter( Filters=[{'Name': 'instance-id', 'Values': ec2instances}])
for instance in instances:
   print(instance.id, instance.instance_type, instance.image_id, instance.public_ip_address)

('i-091dd4eabc9c26ebf', 'm4.large', 'ami-022b9262', '54.218.242.169')
('i-0b3baa39032a638ff', 'm4.large', 'ami-022b9262', '54.218.60.176')


# now we will create the first task definition
we will have two services.
<ul>
<li> The predictor service.   This services reads prediction jobs from the amazon sqs queue service, invokes the predictor to classify it, then it sends the classification, title, the service hostname, the correct answer and a sequence number to the table service. 
<li> the table service.   this is a simple web service that waits for a message from a predictor service and then pushes the result to the aws dynamoDB in table "BookTable"
</ul>
We start with the task definition of the for the table service.  specify that task definition family name, default network and our  AIM role that authorizes the service to use the queue and the dynamoDB.   We are going to deploy this as a docker container which we have saved to the Docker hub. (see the build files for this in directory table-service).  We also need to specify a port binding.

note: the iam arn has been modified and the image is now assumed to be in your account.  (this notebook was run with valid accounts and then edited later for public viewing

In [ ]:
response = client.register_task_definition(
    family='tableservice',
    networkMode='bridge',
    taskRoleArn= 'arn:aws:iam::0123456789012:role/mymicroservices',
            
    containerDefinitions=[
        {
            'name': 'tableservice',
            'image': 'yourdockerhubid/tableservice',
            'cpu': 20,
            'memory': 400,
            'memoryReservation': 123,
            'portMappings': [
                {
                    'containerPort': 8050,
                    'hostPort': 8050,
                    'protocol': 'tcp'
                },
            ],
            'essential': True,
        },
    ],
 )


In [10]:
client.list_task_definitions(familyPrefix='tableservice')['taskDefinitionArns']

[u'arn:aws:ecs:us-west-2:066301190734:task-definition/tableservice:1']

# now we create the tableservice service
note that when we create a task definition it gives it a sequnce number.   
that is because we often revises the task definition during debugging.   We specify that we want at least 50% of our requested instances running at all time.

In [ ]:
response = client.create_service( cluster='cloudbook', 
                                 serviceName='tableservice', 
                                 taskDefinition='tableservice:1', 
                                 desiredCount=2, deploymentConfiguration={
                                    'maximumPercent': 100,
                                    'minimumHealthyPercent': 50 }
)

# now the task definition for the predictor.
note: the iam arn has been modified and the image is now assumed to be in your account

In [20]:
response = client.register_task_definition(
    family='predictor',
    networkMode='bridge',
    taskRoleArn= 'arn:aws:iam::012345678901:role/mymicroservices',
            
    containerDefinitions=[
        {
            'name': 'predictor',
            'image': 'yourdocerhubid/predictor',
            'cpu': 20,
            'memoryReservation': 400,
            'essential': True,
        },
    ],
 )

In [21]:
client.list_task_definitions(familyPrefix='predictor')['taskDefinitionArns']

[u'arn:aws:ecs:us-west-2:066301190734:task-definition/predictor:1',
 u'arn:aws:ecs:us-west-2:066301190734:task-definition/predictor:2',
 u'arn:aws:ecs:us-west-2:066301190734:task-definition/predictor:3',
 u'arn:aws:ecs:us-west-2:066301190734:task-definition/predictor:4']

# create the predictor service
As you can see from the above i am on the 4th iteration of the predictor (earlier versions had a few bugs).  Note that we are creating 8 copies of this service.

In [22]:
response = client.create_service( cluster='cloudbook', 
                                 serviceName='predictor', 
                                 taskDefinition='predictor:4', 
                                 desiredCount=8, deploymentConfiguration={
                                    'maximumPercent': 100,
                                    'minimumHealthyPercent': 50 }
)

now check to see how many services i have and how many tasks.  There should be 10 tasks

In [30]:
client.list_services( cluster='cloudbook')['serviceArns']

[u'arn:aws:ecs:us-west-2:066301190734:service/tableservice',
 u'arn:aws:ecs:us-west-2:066301190734:service/predictor']

In [31]:
client.list_tasks(cluster='cloudbook')['taskArns']

[u'arn:aws:ecs:us-west-2:066301190734:task/3d799728-2394-49a7-8dc3-85360b5290b0',
 u'arn:aws:ecs:us-west-2:066301190734:task/46761ea8-ed02-47c3-9eaa-06439427c94c',
 u'arn:aws:ecs:us-west-2:066301190734:task/5945b74f-697b-4e0c-b441-ca873e818f69',
 u'arn:aws:ecs:us-west-2:066301190734:task/8bd32b4f-6f86-4309-810a-bcf6141ef460',
 u'arn:aws:ecs:us-west-2:066301190734:task/988d5314-99d9-453c-bc8b-bda5fd33726e',
 u'arn:aws:ecs:us-west-2:066301190734:task/9d382215-60ab-4b71-8c18-0970cd278905',
 u'arn:aws:ecs:us-west-2:066301190734:task/9f657c58-f2fd-4bc1-8845-1129a64688e7',
 u'arn:aws:ecs:us-west-2:066301190734:task/c221c15c-9185-4d8e-92ae-b4b048f1158b',
 u'arn:aws:ecs:us-west-2:066301190734:task/e43e3441-3cf5-447d-b8a4-1804a69a09a6',
 u'arn:aws:ecs:us-west-2:066301190734:task/ef8ae905-488b-4a64-840c-7681835bde81']

Now we are done.   Go to the notebook "load_sciml_data_sent_to_aws_queue" to run the data.